<a href="https://colab.research.google.com/github/BATiger/Classic/blob/main/mainf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/MVQA')

import processDataNew as processData
import VGG16
from ques_emb2 import QuestionEmbedding 
from img_emb import ImageEmbedding
#from san1layer import Attention
from san import Attention
import torch
import torch.nn as nn
import torch.nn.functional as F  
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models

use_cuda = True

In [ ]:
data = processData.getAllText('/content/drive/My Drive/Colab Notebooks/MVQA/alldata.json')
testdata = processData.getAllText('/content/drive/My Drive/Colab Notebooks/MVQA/testset.json')
#traindata = processData.getAllText('/content/drive/My Drive/Colab Notebooks/MVQA/trainset.json')
traindata = processData.getAllText('/content/drive/My Drive/Colab Notebooks/MVQA/faulttrainset.json')


questions = processData.getQuestions(data)
answers = processData.getAnswers(data)
imgs_name = processData.get_image_name(data)

dicQuestion = processData.getDic(questions)
ans_dic = list(set(answers))

questions_train = processData.getQuestions(traindata)
imgs_name_train = processData.get_image_name(traindata)
answers_train = processData.getAnswers(traindata)

questions_test = processData.getQuestions(testdata)
imgs_name_test = processData.get_image_name(testdata)
answers_test = processData.getAnswers(testdata)

ques_one_hot_train = []
for question in questions_train:
    one_hot_vector = processData.getVecRep(dicQuestion,question.lower())
    ques_one_hot_train.append(one_hot_vector)



answer_to_vec_train = []
for answer in answers_train:
    if isinstance(answer,str):
          answer = answer.lower()
    vector = processData.getVecRep2(ans_dic,answer)
    answer_to_vec_train.append(vector)

#double the training set
ques_one_hot_train.extend(ques_one_hot_train)
answer_to_vec_train.extend(answer_to_vec_train)
for i in range(len(imgs_name_train)):
      imgs_name_train.append(imgs_name_train[i]+'c')


ques_one_hot_test = []
for question in questions_test:
    one_hot_vector = processData.getVecRep(dicQuestion,question.lower())
    ques_one_hot_test.append(one_hot_vector)

answer_to_vec_test = []
for answer in answers_test:
    if isinstance(answer,str):
        answer = answer.lower()
    vector = processData.getVecRep2(ans_dic,answer)
    answer_to_vec_test.append(vector)

print(len(ans_dic))
print(ans_dic)
print(len(dicQuestion))

517
[2, 4, 'pineal gland', 12, 'non-contrast', 'enhancement of vessels', 'right kidney', 'posteriorly', 'mr - flair', 'irregular', 'hypoxic ischemic injury', 'diffusion weighted imaging (dwi)', 'sharp costophrenic angles', 'sulcal effacement', 'ivc', 'maybe', 'increased opacity in the left retrocardiac region', 't2', 'the surrounding tissue', 'left rectus abdominus', 'semi-upright position', 'ribs', 'sternotomy wires and surgical clips', 'enlarged', 'motion', 'metastases, infection/abcess, glioblastoma', 'abdomen', 'there is massive cerebral hemisphere edema', 'left lung', 'hydrocephalus', '5%', 'right upper lobe', 'abcess', 'sinusitis', 'concave', 'flair', 'ring-enhancing lesions', 'cecum', 'x-ray plain film', 'just one', 't2-weighted', 'pa', 'right pica', 'left temporal horn', 'not sure.', 'multilobulated', 'contrast in the intestines', 'cxr', 'plain film xray', 'bilateral frontal lobes', 'mr - t2 weighted', 'ring enhancing lesion', 't5', 'upper lobes', 'ct with gi and iv contrast', 

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0].split('/')[-1]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


img_transforms = transforms.Compose([
    transforms.ToTensor(),                                     
])


imgdir = 'drive/My Drive/Colab Notebooks/MVQA/VQA-RAD'
imgdataset = ImageFolderWithPaths(imgdir, transform=img_transforms)
imgdataloader = torch.utils.data.DataLoader(imgdataset, batch_size=1, shuffle=False)

vgg2 = VGG16.VGG16(num_classes=10)

#comment this line to cancel pre-training
vgg2.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/vggmodel.pt'))

vgg2.cuda()
vgg2.eval()

dic = {}

for batch in imgdataloader:
    
    img,label,path = batch
    img = Variable(img).cuda()
    
    count = 0
    for layer in vgg2.features.features:
        img = layer(img)
        count = count+1
        if count == 30:
            break
    
    img = img.view(-1, 512, 196).transpose(1, 2)
    img_feature_numpy = img.detach().cpu().numpy() 
    dic[path[0].split('.')[0]] = img_feature_numpy


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [ ]:
que_model = QuestionEmbedding(len(dicQuestion),500,1024,1024,2,0.4,2248,1)
img_model = ImageEmbedding(hidden_size=1024)
attention_model = Attention(1024,512,196,517,0.5)

In [ ]:
if torch.cuda.is_available():
    que_model.cuda()
    img_model.cuda()
    attention_model.cuda()
    print('gpu used!')
    
criterion = nn.CrossEntropyLoss()

optimizer_parameter_group = [
                             {'params': que_model.parameters()},
                             {'params': img_model.parameters()},
                             {'params': attention_model.parameters()}
                            ]

#optimizer = torch.optim.SGD(optimizer_parameter_group,
#                                lr=0.0004,
#                                momentum=0.9)

optimizer = torch.optim.Adam( optimizer_parameter_group,
    		                      eps=1e-8,
		                          lr=0.0004)

gpu used!


In [ ]:
epochs = 20

#training
all_loss_store = []
loss_store = []

for epoch in range(epochs):
    print('epoch: ',epoch)
    running_loss = 0.0
    right_answer = 0
    for i in range(len(ques_one_hot_train)):
      question = torch.tensor(np.array(ques_one_hot_train[i], dtype=np.float32))
      ans = torch.tensor(np.array([answer_to_vec_train[i]],dtype=np.long))
      image = torch.tensor(dic[imgs_name_train[i]])
      if torch.cuda.is_available():
          image = image.cuda()
          question = question.cuda()
          ans = ans.cuda()

          optimizer.zero_grad()
          img_emb = img_model(image)
          ques_emb  =que_model(question)
          #print(img_emb.shape)
          #print(ques_emb.shape)
          output = attention_model(ques_emb, img_emb)
          
          pred = torch.max(output, 1)[1]
          if pred == ans:
             right_answer = right_answer + 1

          loss = criterion(output,ans)

          all_loss_store.append(loss.item())
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
    loss_store.append(running_loss)
    print('training loss:',(running_loss/len(ques_one_hot_train)))
    print('training accuracy:',(right_answer/len(ques_one_hot_train)))
 

epoch:  0


/content/drive/My Drive/Colab Notebooks/MVQA/san.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p1 = self.sf(h1_emb.view(-1, self.img_seq_size)).view(B, 1, self.img_seq_size)
/content/drive/My Drive/Colab Notebooks/MVQA/san.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p2 = self.sf(h2_emb.view(-1, self.img_seq_size)).view(B, 1, self.img_seq_size)


training loss: 3.088079213045596
training accuracy: 0.38609660574412535
epoch:  1
training loss: 1.8502498159364995
training accuracy: 0.529210182767624
epoch:  2
training loss: 1.1908419923589373
training accuracy: 0.679177545691906
epoch:  3
training loss: 0.7145925038165897
training accuracy: 0.8018929503916449
epoch:  4
training loss: 0.44828213817456997
training accuracy: 0.8723890339425587
epoch:  5
training loss: 0.3359221456178486
training accuracy: 0.9060052219321149
epoch:  6
training loss: 0.23845329176508104
training accuracy: 0.9308093994778068
epoch:  7
training loss: 0.21381870069718548
training accuracy: 0.9378263707571801
epoch:  8
training loss: 0.18283734935537643
training accuracy: 0.9454960835509139
epoch:  9
training loss: 0.16303426151798536
training accuracy: 0.9525130548302873
epoch:  10
training loss: 0.1507927653800414
training accuracy: 0.9536553524804178
epoch:  11
training loss: 0.14960554973574905
training accuracy: 0.9559399477806788
epoch:  12
training 

In [ ]:
que_model.eval()
img_model.eval()
attention_model.eval()

print(len(questions_test))
print('test!')
if torch.cuda.is_available():
    right_answer_test = 0
    right = []
    for i in range(len(questions_test)):
      question = torch.tensor(np.array(ques_one_hot_test[i], dtype=np.float32))
      ans = torch.tensor(np.array([answer_to_vec_test[i]],dtype=np.long))
      image = torch.tensor(dic[imgs_name_test[i]])
      if torch.cuda.is_available():
          image = image.cuda()
          question = question.cuda()
          ans = ans.cuda()

          img_emb = img_model(image)
          ques_emb  =que_model(question)

          output = attention_model(ques_emb, img_emb)
          
          pred = torch.max(output, 1)[1]
          if pred == ans:
             right_answer_test = right_answer_test + 1
             right.append(1)
          else:
             right.append(0)

    print('test accuracy:',(right_answer_test/len(questions_test)))
    print(right)
    print(right_answer_test)
    

451
test!


/content/drive/My Drive/Colab Notebooks/MVQA/san.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p1 = self.sf(h1_emb.view(-1, self.img_seq_size)).view(B, 1, self.img_seq_size)
/content/drive/My Drive/Colab Notebooks/MVQA/san.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p2 = self.sf(h2_emb.view(-1, self.img_seq_size)).view(B, 1, self.img_seq_size)


test accuracy: 0.5609756097560976
[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,

In [ ]:
#import os
#model_dir = '/content/drive/My Drive/Colab Notebooks/'
#torch.save(que_model.state_dict(), os.path.join(model_dir, 'question_model.pkl'))
#torch.save(img_model.state_dict(), os.path.join(model_dir, 'image_model.pkl'))
#torch.save(attention_model.state_dict(), os.path.join(model_dir, 'attention_model.pkl'))